In [1]:
import sqlite3
import pandas as pd
import numpy as np

con = sqlite3.connect('../summer')
cur = con.cursor()

In [2]:
TAs = ['\'Mark\'', '\'vhil\'', '\'record\'', '\'Eugy\'']
week1_3 = pd.read_sql_query("""SELECT PlayerName, 
HeadPos_x, HeadPos_y, HeadPos_z, 
LeftHandPos_x, LeftHandPos_y, LeftHandPos_z,
RightHandPos_x, RightHandPos_y, RightHandPos_z,
HeadRot_w, HeadRot_x, HeadRot_y, HeadRot_z,
LeftHandRot_w, LeftHandRot_x, LeftHandRot_y, LeftHandRot_z,
RightHandRot_w, RightHandRot_x, RightHandRot_y, RightHandRot_z
FROM PersonTime
WHERE PlayerName NOT IN ({})
AND RecordingName = 'week1_3'
""".format(",".join(TAs)), con)

week_1_playernames = week1_3['PlayerName'].unique()
print(week_1_playernames)

['Alonzo' 'Dax' 'Emily' 'Isabelle' 'Jake' 'Philip' 'Rishab' 'Sam' 'justin'
 'Alex' 'Jordan']


In [9]:
playerIdx = {
    'Alonzo': 0,
    'Dax': 1,
    'Emily': 2,
    'Isabelle': 3,
    'Jake': 4,
    'Philip': 5,
    'Rishab': 6,
    'Sam': 7,
    'justin': 8,
    'Alex': 9,
    'Jordan': 10
}

motion_input = []
motion_output = []

idx = 0
features = []
for index, row in week1_3.iterrows():
    if idx == 29:        
        motion_input.append(features)
        features = []
        
        output = np.zeros(len(week_1_playernames))
        output[playerIdx[row['PlayerName']]] = 1
        motion_output.append(output)
        
        idx = 0
    
    features.extend((row['HeadPos_x'], row['HeadPos_y'], row['HeadPos_z'],
                     row['LeftHandPos_x'], row['LeftHandPos_y'], row['LeftHandPos_z'],
                     row['RightHandPos_x'], row['RightHandPos_y'], row['RightHandPos_z'],
                     row['HeadRot_w'], row['HeadRot_x'], row['HeadRot_y'], row['HeadRot_z'],
                     row['LeftHandRot_w'], row['LeftHandRot_x'], row['LeftHandRot_y'], row['LeftHandRot_z'],
                     row['RightHandRot_w'], row['RightHandRot_x'], row['RightHandRot_y'], row['RightHandRot_z']))
    idx += 1

training_x = np.asarray(motion_input)
training_y = np.asarray(motion_output)
        
print(training_x.shape)
print(training_y.shape)

(24477, 609)
(24477, 11)


In [3]:
playerIdx = {
    'Alonzo': 0,
    'Dax': 1,
    'Emily': 2,
    'Isabelle': 3,
    'Jake': 4,
    'Philip': 5,
    'Rishab': 6,
    'Sam': 7,
    'justin': 8,
    'Alex': 9,
    'Jordan': 10
}

motion_input = []
motion_output = []
idx = 0
first = week1_3.iloc[0]

# head_delta_x, y, z, lhand_delta_x, y, z, rhand_delta_x, y, z
# total_delta_x, y, z, lhand x y z , rhand x y z
features = []
head_total_x, head_total_y, head_total_z = 0.0, 0.0, 0.0
lhand_total_x, lhand_total_y, lhand_total_z = 0.0, 0.0, 0.0
rhand_total_x, rhand_total_y, rhand_total_z = 0.0, 0.0, 0.0

for index, row in week1_3.iterrows():
    if idx == 59:
        head_delta_x = first['HeadPos_x'] - row['HeadPos_x']
        head_delta_y = first['HeadPos_y'] - row['HeadPos_y']
        head_delta_z = first['HeadPos_z'] - row['HeadPos_z']
        lhand_delta_x = first['LeftHandPos_x'] - row['LeftHandPos_x']
        lhand_delta_y = first['LeftHandPos_y'] - row['LeftHandPos_y']
        lhand_delta_z = first['LeftHandPos_z'] - row['LeftHandPos_z']
        rhand_delta_x = first['RightHandPos_x'] - row['RightHandPos_x']
        rhand_delta_y = first['RightHandPos_y'] - row['RightHandPos_y']
        rhand_delta_z = first['RightHandPos_z'] - row['RightHandPos_z']
        
        motion_input.append([head_delta_x, head_delta_y, head_delta_z,
                         lhand_delta_x, lhand_delta_y, lhand_delta_z,
                         rhand_delta_x, rhand_delta_y, rhand_delta_z,
                         head_total_x, head_total_y, head_total_z,
                         lhand_total_x, lhand_total_y, lhand_total_z,
                         rhand_total_x, rhand_total_y, rhand_total_z
                        ])
        
        output = np.zeros(len(week_1_playernames))
        output[playerIdx[row['PlayerName']]] = 1
        motion_output.append(output)
    
    if idx == 60:
        first = row
        idx = 0
        features = []
        head_total_x, head_total_y, head_total_z = 0.0, 0.0, 0.0
        lhand_total_x, lhand_total_y, lhand_total_z = 0.0, 0.0, 0.0
        rhand_total_x, rhand_total_y, rhand_total_z = 0.0, 0.0, 0.0
    
    head_total_x += row['HeadPos_x']
    head_total_y += row['HeadPos_y']
    head_total_z += row['HeadPos_z']
    lhand_total_x += row['LeftHandPos_x']
    lhand_total_y += row['LeftHandPos_y']
    lhand_total_z += row['LeftHandPos_z']
    rhand_total_x += row['RightHandPos_x']
    rhand_total_y += row['RightHandPos_y']
    rhand_total_z += row['RightHandPos_z']
    idx += 1

training_x = np.asarray(motion_input)
training_y = np.asarray(motion_output)
        
print(training_x.shape)
print(training_y.shape)

(11830, 18)
(11830, 11)


In [4]:
print(training_x.shape)
print(training_y.shape)
print(training_y)

(11830, 18)
(11830, 11)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [13]:
recordings = ['\'week1_3\'', '\'week2_3\'', '\'week3_3\'', '\'week4_3\'', '\'week5_3\'', '\'week6_3\'']
students = ['\'Alonzo\'','\'Dax\'','\'Emily\'','\'Isabelle\'','\'Jake\'','\'Philip\'','\'Rishab\'','\'Sam\'',
            '\'justin\'','\'Alex\'','\'Jordan\'']


all_weeks = pd.read_sql_query("""SELECT PlayerName, 
    HeadPos_x, HeadPos_y, HeadPos_z, 
    LeftHandPos_x, LeftHandPos_y, LeftHandPos_z,
    RightHandPos_x, RightHandPos_y, RightHandPos_z,
    HeadRot_w, HeadRot_x, HeadRot_y, HeadRot_z,
    LeftHandRot_w, LeftHandRot_x, LeftHandRot_y, LeftHandRot_z,
    RightHandRot_w, RightHandRot_x, RightHandRot_y, RightHandRot_z
    FROM PersonTime
    WHERE PlayerName IN ({})
    AND RecordingName IN ({})
    """.format(",".join(students), ",".join(recordings)), con)

print(all_weeks.shape)

(3189974, 22)


In [14]:
def create_labels(row):
    output = np.zeros(len(week_1_playernames))
    output[playerIdx[row['PlayerName']]] = 1
    return output

all_weeks['label'] = all_weeks.apply(lambda x: create_labels(x), axis=1)
